In [1]:
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from tensorflow.keras import datasets

In [14]:
(x, y), _ = datasets.mnist.load_data()

In [15]:
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255 # 0-1
y = tf.convert_to_tensor(y, dtype=tf.int32)

In [16]:
print(x.shape, y.shape)
print(x.dtype, y.dtype)

(60000, 28, 28) (60000,)
<dtype: 'float32'> <dtype: 'int32'>


In [17]:
train_db = tf.data.Dataset.from_tensor_slices((x, y)).batch(128) # 映射x,y 一次取128
train_iter = iter(train_db) # 获得迭代器
sample = next(train_iter)
print('batch:', sample[0].shape, sample[1].shape)

batch: (128, 28, 28) (128,)


In [18]:
# [b, 784] => [b, 256] => [b, 128] => [b, 10]
# [dim_in, dim_out], [dim_out]
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1)) # 梯度爆炸 改成stddev=0.1
b1 = tf.Variable(tf.zeros([256]))
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

In [20]:
lr = 1e-3 # 步长比例

for step, (x, y) in enumerate(train_db):
        # x:[128, 28, 28]
        # y: [128]

        # [b, 28, 28] => [b, 28*28]
        x = tf.reshape(x, [-1, 28 * 28])

        with tf.GradientTape() as tape:  # tf只跟踪tf.Variable
            # x: [b, 28*28]
            # h1 = x@w1 + b1
            # [b, 784]@[784, 256] + [256] => [b, 256] + [256] => [b, 256] + [b, 256]
            h1 = x @ w1 + tf.broadcast_to(b1, [x.shape[0], 256])  # 广播
            h1 = tf.nn.relu(h1)
            # [b, 256] => [b, 128]
            h2 = h1 @ w2 + b2
            h2 = tf.nn.relu(h2)
            # [b, 128] => [b, 10]
            out = h2 @ w3 + b3

            # compute loss
            # out: [b, 10]
            # y: [b] => [b, 10]
            y_onehot = tf.one_hot(y, depth=10)

            # mse = mean(sum(y-out)^2)
            # [b, 10]
            loss = tf.square(y_onehot - out) #  
            # mean: scalar
            loss = tf.reduce_mean(loss)
        
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
#         print('w1', grads[0])
#         print('b1', grads[1])
#         print('w2', grads[2])
#         print('b2', grads[3])
#         print('w3', grads[4])
#         print('b3', grads[5])
        # w1 = w1 - lr * w1_grad
        w1.assign_sub(lr * grads[0]) # assign_sub：原地更新
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])

        if step % 100 == 0:
            print(step, 'loss:', float(loss))


0 loss: 0.5262953042984009
100 loss: 0.20400793850421906
200 loss: 0.2041756808757782
300 loss: 0.18306228518486023
400 loss: 0.1822025328874588
